In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os.path as op
import pickle
from scipy.fft import fft, fftfreq
from tqdm import tqdm

In [37]:
def load_obj(name, path):
    with open(op.join(path, name), 'rb') as f:
        return pickle.load(f)

In [38]:
data_path = "/sps/crnl/pmouches/data/MEG_PHRC_2006_Ap18"
file_prefix = "data_raw_"

### List of all patients

In [39]:
patients = ["25", "27", "38", "61", "83", "97", "18", "22", "54", "68", "69", "84", "105", "42", "20", "30", "11", "47", "56", "101", "117", "4", "8", "26", "66", "67", "79", "92", "111", "2", "24", "65", "73", "100", "118", "119", "23", "35","40", "62", "102", "107", "108", "109"]

In [40]:
data = pd.DataFrame(load_obj(f'{file_prefix}25.pkl',data_path))

In [15]:
data

,meg,events,file,is_reannoted,reannot,detected_spikes
0,"[[3.155443620884047e-28, -1.403105506668496e-1...",[],china_Epi-001_20090710_01.ds,False,[],[]
1,"[[-6.310887241768094e-28, -1.4146868268796115e...","[1759.0, 11486.0]",china_Epi-001_20090710_02.ds,False,[],[]
2,"[[-1.0349855076499675e-27, -6.057839499906236e...","[10990.0, 19761.0, 22084.0, 22570.0]",china_Epi-001_20090710_03.ds,False,[],[]
3,"[[2.145701662201152e-28, -1.0189542581520915e-...","[419.0, 7490.0, 20464.0, 22664.0, 22788.0]",china_Epi-001_20090710_04.ds,False,[],[]
4,"[[-2.398137151871876e-28, -1.6826981959806253e...","[5028.0, 5452.0, 5564.0, 11996.0, 17640.0]",china_Epi-001_20090710_05.ds,False,[],[]
5,"[[3.5025424191812924e-28, -7.908513190185054e-...","[929.0, 1677.0, 1890.0, 4129.0, 17324.0]",china_Epi-001_20090710_06.ds,False,[],[]
6,"[[3.534096855390133e-28, 2.322548986555037e-14...",[11802.0],china_Epi-001_20090710_07.ds,False,[],[]
7,"[[-5.238036410667518e-28, 1.195133903026395e-1...","[299.0, 1342.0, 25983.0]",china_Epi-001_20090710_08.ds,False,[],[]
8,"[[-2.6505726415425997e-28, -3.0911664240707877...","[609.0, 16002.0, 25179.0, 25557.0, 26796.0]",china_Epi-001_20090710_09.ds,True,"[609.0, 16002.0, 25179.0, 25557.0, 26796.0, 18...","[6867.0, 7560.0, 11928.0, 15603.0, 16758.0, 22..."
9,"[[-7.573064690121713e-29, -2.3581885501782516e...","[12043.0, 16845.0, 23535.0]",china_Epi-001_20090710_10.ds,False,[],[]


In [30]:
data["meg"][0][k]

array([-1.38839519e-28,  2.15690050e-13,  4.18981297e-13, ...,
       -3.66874218e-13, -1.19384124e-13, -2.39813715e-28])

In [66]:
# Supposons que data["meg"] soit une liste de matrices NumPy
# Par exemple, data["meg"][i] a une forme (num_channels, num_time_points)
# Le nombre total de canaux est supposé être 274

# Initialisation des paramètres
num_blocks = len(data["meg"])
num_channels = data["meg"][0].shape[0]
num_time_points = data["meg"][0].shape[1]

# Vérifier que tous les blocs ont le même nombre de canaux et de points temporels
for block in data["meg"]:
    assert block.shape[0] == num_channels
    assert block.shape[1] == num_time_points

# Créer un tableau vide pour stocker les données
all_data = np.zeros((num_blocks * num_time_points, num_channels + 2))

# Remplir les colonnes 'is_annoted' et 'block_id'
all_data[:, 0] = False # 'is_annoted' est False (0) pour toutes les lignes
block_ids = np.repeat(np.arange(num_blocks), num_time_points)
all_data[:, 1] = block_ids


# Remplir les données des canaux
for i in range(num_blocks):
    all_data[i * num_time_points:(i + 1) * num_time_points, 2:] = data["meg"][i].T

# Créer les noms des colonnes
column_names = ["is_annoted", "block_id"] + [f"Channel_{i+1}" for i in range(num_channels)]
new_df = pd.DataFrame(all_data, columns=column_names)

for i in range(len(data['reannot'])):
        for event in data['reannot'][i]:
            print(i)
            time_stamp = event//150 + i*num_time_points
            print(time_stamp)
            print(new_df['is_annoted'][time_stamp])
            new_df['is_annoted'][time_stamp] = 1
# print(new_df['is_annoted'].value_counts())



8
216004.0
0.0
8
216106.0
0.0
8
216167.0
0.0
8
216170.0
0.0
8
216178.0
0.0
8
216012.0
0.0
8
216052.0
0.0
8
216109.0
0.0
8
216045.0
0.0
8
216050.0
0.0
8
216104.0
0.0
8
216111.0
0.0
8
216152.0
0.0
8
216167.0
1.0
11
297052.0
0.0
11
297075.0
0.0
11
297077.0
0.0
11
297082.0
0.0
11
297099.0
0.0
11
297152.0
0.0
11
297152.0
1.0
11
297068.0
0.0
11
297082.0
1.0
11
297099.0
1.0
11
297145.0
0.0
11
297172.0
0.0
13
351034.0
0.0
13
351044.0
0.0
13
351063.0
0.0
13
351102.0
0.0
13
351107.0
0.0
13
351107.0
1.0
13
351124.0
0.0
13
351130.0
0.0
13
351138.0
0.0
13
351141.0
0.0
13
351143.0
0.0
13
351162.0
0.0
13
351124.0
1.0


## Window creation

In [67]:
new_df['is_annoted'].value_counts()

is_annoted
0.0    404967
1.0        33
Name: count, dtype: int64

In [88]:
def create_windows_dataframe(data, samples_per_window, nb_samples, nb_channels, sample_rate, overlap_samples):
    windows = []
    
    for i in range(len(data)):
        start = 0
        end = samples_per_window
        while end < nb_samples:
            window_data = {"is_annoted": False, "block_id":i}
            for k in range(nb_channels):
                channel_data = data[i][k, start:end]
                window_data[f"Channel_{k+1}"] = np.array(channel_data)
            window_data["Start_Time"] = start / sample_rate 
            window_data["End_Time"] = end / sample_rate  
            windows.append(window_data)
            start += samples_per_window - overlap_samples
            end = start + samples_per_window
    df = pd.DataFrame(windows)
    return df

In [89]:
def annotate_window(annotations, df):
    for i in range(len(annotations)):
        for event in annotations[i]:
            time_stamp = event/150
            for index, row in df.iterrows():
                if row["Start_Time"] <= time_stamp <= row["End_Time"] and row["block_id"] == i:
                    df.at[index, "is_annoted"] = True
                    
    return df

In [138]:
def window_creation(data):
    data = data[data['is_reannoted'] == True]
    data.reset_index(drop=True, inplace=True)
    nb_channels = data["meg"][0].shape[0]
    nb_samples = data["meg"][0].shape[1]
    sample_rate = 150 # Hz
    window_size = 0.2 # 200 ms
    overlap = 0.03 # 30 ms

    samples_per_window = int(window_size * sample_rate)
    overlap_samples = int (overlap * sample_rate)

    df = create_windows_dataframe(data["meg"], samples_per_window, nb_samples,nb_channels, sample_rate, overlap_samples)
    df = annotate_window(data["reannot"], df)
    return df

## Feature creation

In [13]:
def compute_window_ppa(window):
    return np.max(window) - np.min(window)

In [29]:
def compute_window_upslope(window):
    return np.max(np.diff(window))

In [30]:
def compute_window_downslope(window):
    return np.min(np.diff(window))

In [17]:
def compute_window_std(window):
    return np.std(window)

In [18]:
def compute_window_amplitude_ratio(window):
    return (np.max(window)-np.min(window))/np.mean(window)

In [21]:
def compute_window_sharpness(window):
    slopes = np.diff(window)
    return np.max(np.abs(slopes[1:]-slopes[:-1]))

In [28]:
def compute_window_average_slope(window):
    abs_slopes = np.abs(np.diff(window))
    return np.max((abs_slopes[:-1] + abs_slopes[1:]) / 2)

In [27]:
def compute_window_main_frequency(window):
    n = len(window)
    fft_result = fft(window)
    frequencies = fftfreq(n)
    amplitudes = np.abs(fft_result)
    peak_frequency_index = np.argmax(amplitudes)
    main_frequency = frequencies[peak_frequency_index]
    return main_frequency

In [25]:
def compute_window_phase_congruency(window):
    fft_window = fft(window)
    phases = np.angle(fft_window)
    phase_diff = np.diff(phases)
    phase_congruencies = 1 - (np.abs(phase_diff)/np.pi)
    return np.max(phase_congruencies)

In [33]:
list_of_features = ["ppa", "upslope", "downslope", "std", "amplitude_ratio", "sharpness", "average_slope", "main_frequency", "phase_congruency"]
compute_features = {"ppa" : compute_window_ppa, "upslope" : compute_window_upslope, "downslope": compute_window_downslope, "std": compute_window_std, 
                    "amplitude_ratio": compute_window_amplitude_ratio, "sharpness" : compute_window_sharpness, "average_slope": compute_window_average_slope, "main_frequency" : compute_window_main_frequency,
                    "phase_congruency": compute_window_phase_congruency}

In [35]:
def create_features(df):
    channels = [col for col in df.columns if col.startswith("Channel_")]    
    data_to_df = {}
    for channel in channels:
        channel_features = {feat: [] for feat in list_of_features}
        for window in df[channel]:
            for feat in list_of_features:
                channel_features[feat].append(compute_features[feat](window))
        for feat in list_of_features:
            data_to_df[f'{feat}_{channel}'] = channel_features[feat]
    return pd.DataFrame(data_to_df)

## Preprocessing

In [48]:
from sklearn.preprocessing import  MinMaxScaler

In [63]:
def standardization(df):
    channels = [col for col in df.columns if col.startswith("Channel_")]    
    for channel in channels:
        channel_data = np.array(df[channel].tolist())
        samples_per_window = channel_data.shape[1]
        channel_data = channel_data.reshape(-1)
        mean = np.mean(channel_data)
        std = np.std(channel_data)
        standardized_data = (channel_data - mean) / std
        channel_data = standardized_data.reshape(-1, samples_per_window)
        df[channel] = channel_data.tolist()
    return df

In [98]:
def preprocessing(set):
    y = set['is_annoted']
    standard_scaler = StandardScaler()
    standardized_set = standardization(set)
    X = create_features(standardized_set)
    min_max_scaler = MinMaxScaler()
    X_scaled = min_max_scaler.fit_transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
    return X_scaled_df, y

### Patient splitting

KeyboardInterrupt: 

In [ ]:
patients_preprocessed.keys()

In [92]:
cv = 45

In [93]:
patients_per_fold = len(patients) // cv
patients_per_fold

1

In [94]:
for i in range(0,cv):
    test_patients = patients[i*patients_per_fold:(i+1)*patients_per_fold]
    train_patients = patients[0:i*patients_per_fold] + patients[(i+1)*patients_per_fold:len(patients)]

In [99]:
X_train = None
y_train = None
for i, patient in tqdm(enumerate(test_patients)):
    data = pd.DataFrame(load_obj(f"{file_prefix}{patient}.pkl",data_path))
    patient_df = window_creation(data)
    X_patient_set, y_patient_set = preprocessing(patient_df)                  
    if X_train is None:
        X_train = X_patient_set
    else:
        X_train = pd.concat([X_train, X_patient_set], axis=0, ignore_index=True)

    if y_train is None:
        y_train = y_patient_set
    else:
        y_train = np.hstack([y_train, y_patient_set])

1it [04:24, 264.65s/it]


In [100]:
X_train

,ppa_Channel_1,upslope_Channel_1,downslope_Channel_1,std_Channel_1,amplitude_ratio_Channel_1,sharpness_Channel_1,average_slope_Channel_1,main_frequency_Channel_1,phase_congruency_Channel_1,ppa_Channel_2,...,phase_congruency_Channel_273,ppa_Channel_274,upslope_Channel_274,downslope_Channel_274,std_Channel_274,amplitude_ratio_Channel_274,sharpness_Channel_274,average_slope_Channel_274,main_frequency_Channel_274,phase_congruency_Channel_274
0,0.160524,0.222788,0.805722,0.102598,0.644968,0.262395,0.210722,0.0,0.926153,0.228997,...,0.954570,0.199151,0.263888,0.850253,0.157614,0.493694,0.244362,0.076959,0.0,0.879748
1,0.090270,0.173453,0.833942,0.100155,0.645262,0.231391,0.098121,0.0,0.777259,0.102958,...,0.698447,0.088588,0.189247,0.829944,0.083950,0.493623,0.308499,0.091367,0.0,0.936554
2,0.097036,0.151710,0.890313,0.063707,0.645351,0.260724,0.086457,0.0,0.906634,0.079064,...,0.707060,0.103950,0.263186,0.867689,0.110466,0.493635,0.216077,0.119778,0.0,0.648747
3,0.106168,0.132934,0.841357,0.063393,0.645272,0.357984,0.138910,0.0,0.927108,0.073595,...,0.928150,0.258801,0.297702,0.799613,0.250375,0.493835,0.470797,0.173255,0.0,0.949900
4,0.062134,0.100999,0.855290,0.042347,0.645331,0.204433,0.056099,0.0,0.814962,0.067802,...,0.915549,0.039222,0.248434,0.866109,0.024890,0.493564,0.349350,0.106784,0.0,0.986028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3109,0.150905,0.185605,0.862824,0.127193,0.645098,0.360719,0.169138,0.0,0.962172,0.156260,...,0.893695,0.050708,0.090587,0.864038,0.070395,0.493754,0.147316,0.037551,0.0,0.944663
3110,0.216969,0.185605,0.907955,0.173469,0.644689,0.247707,0.120567,0.0,0.938470,0.305242,...,0.924335,0.282712,0.202560,0.803067,0.301338,0.490776,0.264942,0.207098,0.1,0.890693
3111,0.113178,0.097933,0.906469,0.084487,0.644313,0.213994,0.061255,0.0,0.980849,0.165159,...,0.911924,0.144025,0.234590,0.868272,0.134626,0.492582,0.319391,0.133699,0.0,0.756562
3112,0.069854,0.107059,0.920816,0.053118,0.644997,0.149861,0.060019,0.0,0.895837,0.101703,...,0.828643,0.234240,0.497968,0.794617,0.240920,0.494442,0.322839,0.223392,0.0,0.986354


# SelectKBest

In [103]:
from sklearn.feature_selection import SelectKBest, chi2

In [107]:
selector = SelectKBest(chi2, k=X_train.shape[1])

In [111]:
selected = selector.fit_transform(X_train, y_train)

In [114]:
pd.DataFrame(selected, columns=X_train.columns)

,ppa_Channel_1,upslope_Channel_1,downslope_Channel_1,std_Channel_1,amplitude_ratio_Channel_1,sharpness_Channel_1,average_slope_Channel_1,main_frequency_Channel_1,phase_congruency_Channel_1,ppa_Channel_2,...,phase_congruency_Channel_273,ppa_Channel_274,upslope_Channel_274,downslope_Channel_274,std_Channel_274,amplitude_ratio_Channel_274,sharpness_Channel_274,average_slope_Channel_274,main_frequency_Channel_274,phase_congruency_Channel_274
0,0.160524,0.222788,0.805722,0.102598,0.644968,0.262395,0.210722,0.0,0.926153,0.228997,...,0.954570,0.199151,0.263888,0.850253,0.157614,0.493694,0.244362,0.076959,0.0,0.879748
1,0.090270,0.173453,0.833942,0.100155,0.645262,0.231391,0.098121,0.0,0.777259,0.102958,...,0.698447,0.088588,0.189247,0.829944,0.083950,0.493623,0.308499,0.091367,0.0,0.936554
2,0.097036,0.151710,0.890313,0.063707,0.645351,0.260724,0.086457,0.0,0.906634,0.079064,...,0.707060,0.103950,0.263186,0.867689,0.110466,0.493635,0.216077,0.119778,0.0,0.648747
3,0.106168,0.132934,0.841357,0.063393,0.645272,0.357984,0.138910,0.0,0.927108,0.073595,...,0.928150,0.258801,0.297702,0.799613,0.250375,0.493835,0.470797,0.173255,0.0,0.949900
4,0.062134,0.100999,0.855290,0.042347,0.645331,0.204433,0.056099,0.0,0.814962,0.067802,...,0.915549,0.039222,0.248434,0.866109,0.024890,0.493564,0.349350,0.106784,0.0,0.986028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3109,0.150905,0.185605,0.862824,0.127193,0.645098,0.360719,0.169138,0.0,0.962172,0.156260,...,0.893695,0.050708,0.090587,0.864038,0.070395,0.493754,0.147316,0.037551,0.0,0.944663
3110,0.216969,0.185605,0.907955,0.173469,0.644689,0.247707,0.120567,0.0,0.938470,0.305242,...,0.924335,0.282712,0.202560,0.803067,0.301338,0.490776,0.264942,0.207098,0.1,0.890693
3111,0.113178,0.097933,0.906469,0.084487,0.644313,0.213994,0.061255,0.0,0.980849,0.165159,...,0.911924,0.144025,0.234590,0.868272,0.134626,0.492582,0.319391,0.133699,0.0,0.756562
3112,0.069854,0.107059,0.920816,0.053118,0.644997,0.149861,0.060019,0.0,0.895837,0.101703,...,0.828643,0.234240,0.497968,0.794617,0.240920,0.494442,0.322839,0.223392,0.0,0.986354


In [131]:
feature_scores = {feature: selector.scores_[i] for i, feature in enumerate(X_train.columns)}

In [132]:
feature_mean_scores = {}
for feature in list_of_features:
    scores = []
    for col in X_train.columns:
        if col.startswith(feature):
            scores.append(feature_scores[col])
    feature_mean_scores[feature] = np.mean(scores)

In [133]:
feature_mean_scores = {k: v for k, v in sorted(feature_mean_scores.items(), key=lambda x: x[1], reverse=True)}

In [134]:
feature_mean_scores

{'std': 22.51399165857163,
 'ppa': 22.115812718224586,
 'average_slope': 19.16913981410649,
 'upslope': 11.595183600235803,
 'sharpness': 4.209004305853786,
 'main_frequency': 2.957182935627046,
 'downslope': 2.7994009789415766,
 'phase_congruency': 0.014246868553758662,
 'amplitude_ratio': 0.003881733164152233}